# Video Game Recommendation
The following notebook will demonstrate a visual data analysis of various video game data and a video game recommendation algorithm. Selecting a platform below will show three analyses related to that platform.
After a platform is selected, you may choose a genre, at which point the games fitting into those categories will be listed in the "Game" dropdown box. Selecting a game from this box will recommend an additional game you may like.

In [65]:
%matplotlib widget

import pandas as pd
import matplotlib as mpl
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import ipywidgets as widgets
import IPython.core.display
import IPython.display

def refresh_games_list():
    global games_list
    global games_frame
    global games_dropdown
    global selected_platform
    global selected_genre

    games_frame = videogame_database[(videogame_database["Platform"] == selected_platform) & (videogame_database["Genre"] == selected_genre)]
    games_list = games_frame["Name"].tolist()
    games_list.sort()

    try:
        games_dropdown.options = games_list
    except:
        pass


def platform_change(change):
    global selected_platform
    global selected_genre
    global genre_dropdown_list
    global genre_dropdown

    selected_platform = change["new"]

    selected_platform_frame = videogame_database[videogame_database["Platform"] == selected_platform]
    genre_dropdown_list = selected_platform_frame["Genre"].unique().tolist()
    genre_dropdown_list.sort()

    selected_genre = genre_dropdown_list[0]
    genre_dropdown.options = genre_dropdown_list

    refresh_games_list()
    update_plots()


def genre_change(change):
    global selected_genre

    selected_genre = change["new"]

    refresh_games_list()


def game_change(change):
    global games_frame
    global recommended_games_list
    global recommended_games_dropdown

    selected_game_row = games_frame[games_frame["Name"] == change["new"]]
    vectorizer = CountVectorizer()

    # Create developer name similarity vector.
    developer_vocabulary = vectorizer.fit(selected_game_row["Developer"])
    developer_selected_vector = developer_vocabulary.transform(selected_game_row["Developer"])
    developer_similarity_vector = developer_vocabulary.transform(games_frame["Developer"])

    # Create game name similarity vector.
    name_vocabulary = vectorizer.fit(selected_game_row["Name"])
    name_selected_vector = name_vocabulary.transform(selected_game_row["Name"])
    name_similarity_vector = name_vocabulary.transform(games_frame["Name"])

    # Create critic score similarity vector.
    critic_score_selected_vector = selected_game_row["Critic_Score"]
    critic_score_similarity_vector = games_frame["Critic_Score"]

    critic_score_selected_vector = np.array(critic_score_selected_vector)
    critic_score_similarity_vector = np.array(critic_score_similarity_vector)

    # Create user score similarity vector.
    user_score_selected_vector = selected_game_row["User_Score"]
    user_score_similarity_vector = games_frame["User_Score"]

    user_score_selected_vector = user_score_selected_vector.replace(to_replace="tbd", value="0")
    user_score_similarity_vector = user_score_similarity_vector.replace(to_replace="tbd", value="0")

    user_score_selected_vector = np.array(user_score_selected_vector)
    user_score_similarity_vector = np.array(user_score_similarity_vector)

    user_score_selected_vector = user_score_selected_vector.astype("float64")
    user_score_similarity_vector = user_score_similarity_vector.astype("float64")

    #print(critic_score_selected_vector)
    #print(critic_score_similarity_vector)

    # Determine the similarity of all characteristics.
    developer_cosine = cosine_similarity(developer_selected_vector, developer_similarity_vector)
    name_cosine = cosine_similarity(name_selected_vector, name_similarity_vector)
    # Invert critic score difference and scale to cosine similarities.
    critic_score_similarity = ((100 - np.absolute(critic_score_selected_vector - critic_score_similarity_vector)) / 100)
    user_score_similarity = ((10 - np.absolute(user_score_selected_vector - user_score_similarity_vector)) / 10)

    similarity_vector = developer_cosine + name_cosine + critic_score_similarity + user_score_similarity
    #print(similarity_vector)

    #print (selected_game_vector.toarray())
    #similarity_vector = cosine_similarity(selected_game_vector, test_games_vector)
    #similarity_frame = pd.DataFrame(similarity_vector[0], columns=["Similarity"])
    recommended_games_frame = pd.DataFrame(games_frame["Name"])
    recommended_games_frame.insert(1, "Similarity", similarity_vector[0])
    recommended_games_frame.sort_values(by="Similarity", ascending=False, inplace=True)
    print (recommended_games_frame)
    #print(similarity_vector[0])
    recommended_games_list = recommended_games_frame["Name"].tolist()
    # Remove the currently selected game from the list.
    recommended_games_list.remove(change["new"])
    # Limit list to 5 games.
    recommended_games_list = recommended_games_list[:5]
    recommended_games_dropdown.options = recommended_games_list
    #print(similarity_vector.tolist())
    #print(pd.DataFrame(similarity_vector, columns=["Similarity"]))
    #print (games_frame)
    #print (selected_game_row)
    #print (developer_vocabulary.
    #print(characteristics_frame)


def update_plots():
    global videogame_database
    global selected_platform
    global output
    global platform_dropdown
    global genre_dropdown
    global games_dropdown
    global recommended_games_dropdown
    global figure1
    global figure2
    global figure3

    IPython.display.clear_output()
    
    display(platform_dropdown)
    display(genre_dropdown)
    display(games_dropdown)
    display(recommended_games_dropdown)

    with output:
        output.clear_output()

        mpl.pyplot.close(1)
        mpl.pyplot.close(2)
        mpl.pyplot.close(3)
        
        print("This bar chart shows the average global sales among games on this platform by each genre available.")
        figure1 = mpl.pyplot.figure(figsize=(10,7))
        print("This pie chart shows the percentage of games in each genre for this platform.")
        figure2 = mpl.pyplot.figure(figsize=(10,10))
        print("This scatter plot shows possible correlations between critic scores and global sales.")
        figure3 = mpl.pyplot.figure(figsize=(10,5))

        selected_platform_frame = videogame_database[videogame_database["Platform"] == selected_platform]
        genre = selected_platform_frame["Genre"].unique().tolist()
        genre.sort()

        # Calculate the average global sales for each platform.
        mean_global_sales = selected_platform_frame.sort_values("Genre").groupby(["Genre"])["Global_Sales"].mean()
        sales = mean_global_sales.tolist()

        title = "Average Global Sales by Genre on Platform: {0} (In Millions)".format(selected_platform)

        mpl.pyplot.figure(1)
        mpl.pyplot.barh(genre, sales, color ='blue', align="edge")
        mpl.pyplot.title(title)
        mpl.pyplot.xlabel("Sales")
        mpl.pyplot.ylabel("Genre")

        # Calculate the percentage of games in each genre on the platform.
        title = "Percentage of Games in Genre on Platform: {0}".format(selected_platform)

        genre_percentage = selected_platform_frame.sort_values("Genre").groupby(["Genre"])["Name"].count()

        mpl.pyplot.figure(2)
        mpl.pyplot.pie(genre_percentage, None, genre)
        mpl.pyplot.title(title)
        mpl.pyplot.legend(loc="right")

        # Calculate the scatter of global sales and critic scores on the platform.
        sales = selected_platform_frame["Global_Sales"]
        scores = selected_platform_frame["Critic_Score"]
        selected_platform_frame = selected_platform_frame.replace(to_replace="tbd", value="0")
        selected_platform_frame = selected_platform_frame.fillna(0)
        #scores = selected_platform_frame["User_Score"].astype(float)
        #scores = scores.multiply(10)
        #scores = scores.astype(int)

        title = "Distribution of Sales over Critic Score on Platform: {0}".format(selected_platform)

        mpl.pyplot.figure(3)
        mpl.pyplot.scatter(scores, sales)
        mpl.pyplot.title(title)
        mpl.pyplot.xlabel("Scores")
        mpl.pyplot.ylabel("Sales")
    
    display(output)

recommended_games_list=["None"]

videogame_database = pd.read_csv("Video_Games_Sales_as_at_22_Dec_2016.csv")

platform_dropdown_list = videogame_database["Platform"].unique().tolist()

selected_platform = platform_dropdown_list[0]
selected_platform_frame = videogame_database[videogame_database["Platform"] == selected_platform]

genre_dropdown_list = selected_platform_frame["Genre"].unique().tolist()
genre_dropdown_list.sort()

selected_genre = genre_dropdown_list[0]

platform_dropdown = widgets.Dropdown(options=platform_dropdown_list, description="Platform:", value=selected_platform)
genre_dropdown = widgets.Dropdown(options=genre_dropdown_list, description="Genre:", value=selected_genre)

refresh_games_list()
games_dropdown = widgets.Dropdown(options=games_list, description="Game:", value=games_list[0])
recommended_games_dropdown = widgets.Dropdown(options=recommended_games_list, description="Recommended:", value=recommended_games_list[0])

platform_dropdown.observe(platform_change, names = "value")
genre_dropdown.observe(genre_change, names = "value")
games_dropdown.observe(game_change, names = "value")
# Do initial output setup.
output = widgets.Output()
    
update_plots()



#similarity_characteristics = ["Developer", "Critic_Score", "Critic_Count", "User_Score", "User_Count"]
#characteristics_frame = games_frame[similarity_characteristics]

#selected_game_row = games_frame[games_frame["Name"] == change["new"]]
#selected_game_row = games_frame[games_frame["Name"] == "Grand Theft Auto IV"]
#vectorizer = CountVectorizer()

#developer_vocabulary = vectorizer.fit(selected_game_row["Developer"])
#selected_game_vector = developer_vocabulary.transform(selected_game_row["Developer"])
#test_games_vector = developer_vocabulary.transform(games_frame["Developer"])
#print (selected_game_vector.toarray())
#similarity_vector = cosine_similarity(selected_game_vector, test_games_vector)
#similarity_frame = pd.DataFrame(similarity_vector[0], columns=["Similarity"])
#recommended_games_frame = pd.DataFrame(games_frame["Name"])
#recommended_games_frame.insert(1, "Similarity", similarity_vector[0])
#recommended_games_frame.sort_values(by="Similarity", ascending=False, inplace=True)
#print(similarity_vector[0])
#print (recommended_games_frame)
#print(similarity_vector.tolist())
#print(pd.DataFrame(similarity_vector, columns=["Similarity"]))
#print (games_frame)
#print (selected_game_row)
#print (developer_vocabulary.
#print(characteristics_frame)


#selected_game_row = games_frame[games_frame["Name"] == "Grand Theft Auto IV"]
#vectorizer = CountVectorizer()

#developer_vocabulary = vectorizer.fit(selected_game_row["Developer"])
#selected_developer_vector = developer_vocabulary.transform(selected_game_row["Developer"])
#developer_similarity_vector = developer_vocabulary.transform(games_frame["Developer"])
#similarity = cosine_similarity(selected_developer_vector, developer_similarity_vector)

#name_vocabulary = vectorizer.fit(selected_game_row["Name"])

# Create developer name similarity matrix.
#print(similarity[0])

Dropdown(description='Platform:', options=('X360', 'PS3', 'PS', 'DC', 'PS2', 'PS4', 'Wii', 'XB', 'XOne', 'GC',…

Dropdown(description='Genre:', options=('Action', 'Adventure', 'Fighting', 'Misc', 'Platform', 'Puzzle', 'Raci…

Dropdown(description='Game:', options=('007: Quantum of Solace', 'Ace Combat: Assault Horizon', 'Adventure Tim…

Dropdown(description='Recommended:', options=('Harry Potter and the Order of the Phoenix', 'The Lord of the Ri…

Output()

                                                Name  Similarity
259                              Assassin's Creed II    4.000000
342                    Assassin's Creed: Brotherhood    3.756497
1781                   Assassin's Creed: Revelations    3.556497
683                  Assassin's Creed IV: Black Flag    3.393603
963                             Assassin's Creed III    3.253603
...                                              ...         ...
5817  Bakugan Battle Brawlers: Defenders of the Core    0.840000
8073                         Jumper: Griffin's Story    0.840000
6329                       The Spiderwick Chronicles    0.810000
6698                                     Ratatouille    0.780000
7054                                       Planet 51    0.750000

[245 rows x 2 columns]
                                                Name  Similarity
963                             Assassin's Creed III    4.000000
1781                   Assassin's Creed: Revelations    3.726497
6